In [1]:
from lib.util import *
from lib.option_pricing import *

# Optimal Exercise of American Options

The Black-Scholes equation is:

$$\frac{\partial f}{\partial t} + r S_t \frac{\partial f}{\partial S_t} + \frac{1}{2}\sigma^2 S_t^2 \frac{\partial ^2 f}{\partial S_t^2} = r f(S_t, t)$$

For Europen puts, the boundary conditions are $P(\infty,t) = 0; \, P(S_t, T) = \max(0, K-S_T)$

For Europen calls, the boundary conditions are $C(0,t) = 0; \, C(S_t, T) = \max(0, S_T-K); \, C(S_t, t) \xrightarrow[S_t \rightarrow \infty]{} S_t$

Solving yields 
\begin{align*}
P(S_t, t) &= -S_t \Phi(-d_1) + K e^{-r(T-t)}\Phi(-d_2) \\
C(S_t, t) &= S_t \Phi(d_1) - K e^{-r(T-t)}\Phi(d_2)
\end{align*}

where 
\begin{align*}
d_1 &= \frac{\ln(S_t/K) + (r+\sigma^2/2)(T-t)}{\sigma\sqrt{T-t}} \\
d_2 &= \frac{\ln(S_t/K) + (r-\sigma^2/2)(T-t)}{\sigma\sqrt{T-t}} \\ 
    &= d_1 - \sigma\sqrt{T-t}
\end{align*}
and $\Phi$ is the standard normal cdf.

In [2]:
args, num_steps = (21, 20, 0.1, 0.25, 0.2), 50
bs = Black_Scholes(*args)
bl = Binomial_lattice(*args, num_steps)

###### European options

In [3]:
print("Black-Scholes    prices:", bs.eur_call_price(), bs.eur_put_price())
print("Binomial Lattice prices:", bl.eur_call_price(), bl.eur_put_price())

Black-Scholes    prices: 1.7689254137361274 0.3728988798712338
Binomial Lattice prices: 1.7724880661245208 0.3764615322595367


In [4]:
tol = 0.01
print("Black-Scholes and Binom Lattice call check:", np.isclose(bs.eur_call_price(), bl.eur_call_price(), tol))
print("Black-Scholes and Binom Lattice  put check:", np.isclose(bs.eur_put_price(), bl.eur_put_price(), tol))

Black-Scholes and Binom Lattice call check: True
Black-Scholes and Binom Lattice  put check: True


###### American options

In [5]:
print("Binomial Lattice prices:", bl.am_call_price(), bl.am_put_price())

Binomial Lattice prices: 1.7724880661245208 0.39239194360285884


In [6]:
print("Heuristically, no early exercise of call option:", bool(1 - bl.am_call_early_exercise()))
print("Early exercise of put option:                   ", bl.am_put_early_exercise())

Heuristically, no early exercise of call option: True
Early exercise of put option:                    True


###### Longstaff-Schwartz

In [7]:
from lib.longstaff_schwartz import *

###### Alternative to LS

MDP formulation:

- State is $[t, H_t]$, where $t$ is the current time and $H_t$ the history of underlyer prices
- Action is to exercise or not to exercise, that is the question
- Reward is 0 if action is not to exercise, and is payoff if action is to exercise
- Discount factor $\gamma$
- $P$ governed by underlyer price movements